In [1]:
##Imports 
import sys
import os.path
import urllib.request
from joblib import Parallel, delayed
import tarfile
from MNIST_sequence import MNIST_funcs
import joblib
from MNIST_sequence.MNIST_funcs import stream_normalise_mean_and_range as snmr
from MNIST_sequence.MNIST_funcs import stream_normalise_mean_and_std as snms
import esig


## Generating Sequential MNIST data

The aim here is to generate MNIST data using score based diffusion models
The proposed steps to doing this are 
- Obtain signatures from MNIST dataset
- Add noise to signatures
- Train NN to denoise signatures to recover original signatures
- Sample from random noise and denoise to obtain new MNIST data
- Evaluate this data

## Step 1: Load in Data

In [ ]:
##import sequence MNIST
host_path = "https://raw.githubusercontent.com/edwin-de-jong/mnist-digits-stroke-sequence-data/master/sequences.tar.gz"
seq_MNIST_path = "seq_MNIST" #local path to store the data

def get_seq_MNIST(seq_MNIST_host_path = host_path, loc_path = seq_MNIST_path):
    os.makedirs(seq_MNIST_path,exist_ok=True)
    tgz_file = "seq_MNIST.tgz"
    urllib.request.urlretrieve(host_path,tgz_file)
    seq_MNIST_tgz = tarfile.open(tgz_file)
    seq_MNIST_tgz.extractall(path=seq_MNIST_path)
    seq_MNIST_tgz.close() 

get_seq_MNIST()

## Step 2: we extract the signatures

In [ ]:
#Extract sequences
N_CPU = joblib.cpu_count()
train_points, train_inputs, train_targets = MNIST_funcs.mnist_train_data(seq_MNIST_path, N_CPU)
test_points, test_inputs, test_targets = MNIST_funcs.mnist_test_data(seq_MNIST_path, N_CPU)

compute signatures

In [ ]:
SIGNATURE_LEVEL = 8

train_sigs_snmr = Parallel(n_jobs=N_CPU)([delayed(esig.stream2sig)(snmr(train_points[k]), SIGNATURE_LEVEL)
                                          for k in range(len(train_points)) ])    
test_sigs_snmr = Parallel(n_jobs=N_CPU)([delayed(esig.stream2sig)(snmr(test_points[k]), SIGNATURE_LEVEL)
                                         for k in range(len(test_points)) ])
    
train_sigs_snms = Parallel(n_jobs=N_CPU)([ delayed(esig.stream2sig)(snms(train_points[k]), SIGNATURE_LEVEL)
                                          for k in range(len(train_points)) ])    
test_sigs_snms = Parallel(n_jobs=N_CPU)([ delayed(esig.stream2sig)(snms(test_points[k]), SIGNATURE_LEVEL)
                                         for k in range(len(test_points)) ])